In [ ]:
import requests
import math
from time import sleep
import numpy as np
import pandas as pd

In [ ]:
headers = {
    'referer': 'https://pcmap.place.naver.com/place/31505829?entry=pll&from=nx&fromNxList=true&from=map&fromPanelNum=2&ts=1651655247936',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'
}

In [ ]:
#워터파크 이름

review_data = pd.DataFrame(data=[], columns=['별점','리뷰'])

total_item_no = None

now_loop_count = 0
total_loop_count = 10000
total_item = 0

while True:
    if total_loop_count<=now_loop_count:
        break
    data = dict(
    operationName= "getVisitorReviews",
    query="query getVisitorReviews($input: VisitorReviewsInput) { visitorReviews(input: $input) { items { id rating author { id nickname from imageUrl objectId url review { totalCount imageCount avgRating __typename } theme { totalCount __typename } __typename } body thumbnail media { type thumbnail class __typename } tags status visitCount viewCount visited created reply { editUrl body editedBy created replyTitle __typename } originType item { name code options __typename } language highlightOffsets apolloCacheId translatedText businessName showBookingItemName showBookingItemOptions bookingItemName bookingItemOptions votedKeywords { code iconUrl iconCode displayName __typename } userIdno isFollowing followerCount followRequested loginIdno __typename } starDistribution { score count __typename } hideProductSelectBox total showRecommendationSort __typename } }",
    variables=dict(
        id="31505829",
        input={
            "bookingBusinessId": "57137",
            "businessId": "31505829",
            "businessType": "place",
            "cidList": ["222117","222121","222161","222160","232110"],
            "display": 20,
            "getAuthorInfo": True,
            "includeContent": True,
            "includeReceiptPhotos": True,
            "isPhotoUsed": False,
            "item": "0",
            "page": now_loop_count+1
            }
        )
    )
    resp = requests.post('https://pcmap-api.place.naver.com/place/graphql',
                     headers=headers,
                     json=data)
    data = resp.json()
    
    total_item = data['data']['visitorReviews']['total']
    if total_item_no is None and total_item:
        # 첫 시작 & 요청 제대로
        total_item_no = total_item
        total_loop_count = math.ceil(total_item_no/10)
        now_loop_count+=1
        
        print("첫시작 응답 잘 옴")
        print("total개수;", total_item_no)
        
    elif total_item_no is None and not total_item:
        # 요청 제대로 안 옴 & 첫 시작 
        print(" 요청 제대로 안 옴 & 첫 시작  => 기다림")
        sleep(3)
        continue
        
        
    elif not total_item and not total_item_no:
        # 첫 시작 X & 응답 안줌 => 기다림
        sleep(3)
        print("첫 시작 X & 응답 안줌 => 기다림")
        continue
        
    elif total_item_no and total_item:
        # 첫 시작 X  & 응답도 잘 옴
        now_loop_count+=1
        sleep(1)
        print("응답 잘 옴 ")
    else:
        print("첫 시작 X & 응답 안줌")
        print("total item: ", total_item, "\n total_item_no:", total_item_no)
        sleep(3)
        continue
    
    items = data['data']['visitorReviews']['items']
    print("{}페이지 items개수 {}".format(now_loop_count, len(items)))
    for item in items:
        tmp = []
        tmp.append(str(item['rating']) + " " + str(item['created']))
        tmp.append(item['body'])
        tmp = pd.DataFrame(data=[tmp], columns=review_data.columns)
        review_data = pd.concat([review_data,tmp])
        

In [ ]:
review_data

,별점,리뷰
0,None 4.20.수,아쿠아바 테이블아래물만 제외하고는 어디하나 청결하지 않은곳이 없었어요. 온가족. 너...
0,None 4.20.수,인생샷 남기기 정말 좋은 스파수영장이네요 주말 9시 30분 입장했는데 10팀 정도 ...
0,None 3.9.수,스파는 풀이 제법 있는 편인데 스파와 사우나 모두 습/건식사우나가 운영되지 않는 점...
0,None 4.13.수,추울때도 매력있었는데 따뜻한날도 넘 좋네요!! 실내는 공기가 좋지 못한데 야외 진짜...
0,None 4.19.화,생각보다 규모는 작지만 자연을 즐길 수 있는 온천이라 너무 좋았어요!
...,...,...
0,5 19.8.24.토,잘놀았어요
0,3 20.2.13.목,잘 보내고 갑니다. 다만 물이 별로 따뜻하진 않네요.
0,5 19.8.24.토,완전 너무 좋았어요~~\n힐링 제대로 하고왔습니다:)
0,4 19.8.24.토,좋았습나다


In [ ]:
review_data.to_csv('포레스트리솜해브나인스파.csv', header= False, index =False)